<a href="https://colab.research.google.com/github/AdmiralHonda/ml_intro/blob/main/ch02/mecab%E3%81%AE%E3%82%A4%E3%83%B3%E3%82%B9%E3%83%88%E3%83%BC%E3%83%AB%E3%81%A8%E4%BD%BF%E3%81%84%E6%96%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 便利ツールの紹介

ここでは言語処理を行う上で欠かせないツールについて紹介します。　　

### 目次
- MeCab
- Pandas

## MeCab

### 文章の分かち書き
> 文章を単語の集合に変換  
> 分割された各単語には品詞などの情報が付与
> 内部で保持している辞書を見て分けている（辞書に大きく依存）

日本語の文章は欧米のように単語間がスペースで区切られていないので、どういった単語の集合なのかを知るために単語単位で分割する必要があります。  
このツールで目指すのは、  
`"文章"　⇒　["単語","単語",...,"単語"]`  
のように最終的には単語のリスト（文字列のリスト）に変換します。    

### 分析の前処理に
> 多くの分析ツール、機械学習のモデルでは文章を単語のリストとしたものを入力とする  

### データ分析
> ただ分割するだけではなく、品詞などもわかる
> 品詞の分布などで考察もできる


### 使うには
> 作成者がコマンドツールとして提供  
> コマンドを各プログラミング言語で実行し、結果をまた言語内で読み込める形式にしたライブラリが存在
> PythonではMeCab(mecab-python)が存在


1.   mecabのインストール（パッケージ管理やgitのソースから）
2.   各言語でmecabを使うライブラリのインストール、または自作
3.   必要なら目的の処理に適した辞書のインストール

元ソース : https://taku910.github.io/mecab/

In [ ]:
# @title 下準備（ubuntu環境なのでaptでインストールしてます）
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null # mecabの利用に必要なライブラリのインストール
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null                    # gitから辞書ファイルのクローン
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1                   # クローンした辞書のインストール
!pip install mecab-python3==0.7 > /dev/null                                                             # 0.7意外だと謎のエラーが発生して安定しない

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc                                                              # 辞書の参照先にインストール先のディレクトリを追加
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"                                                    # mecabの設定ファイルに新しく辞書を追加したことを追記

In [ ]:
"""
MeCabを使用するオブジェクトの初期化
"""
import MeCab
m = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')                         # mecabはコマンドでは -d の後に追加した辞書のPATHを指定して追加の辞書を扱う
                                                                                                        # MeCab.Taggerには主にコマンドのオプションを文字列で渡す。（なんもない場合はMeCab.Tagger()でもいい）

In [ ]:
"""
基本的にparseToNodeで単語を分割し,返り値のnodeに一個目の単語が入ります。  
二個目以降はnode.nextで参照します。
データが連結リストになっています。
また各nodeは様々なメンバ変数を持っています。例）surfaceは見出し語
また、featuresで,区切りの文字列として解析結果を表示できます。
"""
node = m.parseToNode("そうして自身よりも遥かな高みにいるコムギとの対局を続けていくうちに自らの価値観に変化が生じ始め戸惑いを抱く。")

print("nodeの型：",type(node),"\n")

while node:                                                                                             # 条件がnodeはNoneでないときに表示するのでwhileで回す方が楽
    #単語を取得
    word = node.surface                                                                                 # これで見出し語だけ抽出できます
    #品詞を取得
    pos = node.feature.split(",")                                                                       # 品詞や見出し語などの情報を文字列のリストに変換
    print('{0} , {1}'.format(word, pos))
    #次の単語に進める
    node = node.next

In [ ]:
"""
学習データ生成の例
"""
sample_corpas = [\
                 "月末に菊間先生も誘って研究室で飲み会をします。多分費用は掛かりませんw",\
                 "この前合コンで大失敗しました。",\
                 "話を途切れさせてはまずいと焦って自分の話しかしなかったのがいけなかったな",\
                 "橋本君合コンの設定とかしてくれない？"]
print("【生の学習データの例】:\n\n",sample_corpas)

import MeCab

m = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -Owakati')                # -Owakatiというオプションで分かち書き(区切りはスペース)した文書を出力 

output_corpus = ""                                                                                      # 最終的に出力される学習データ
for sentence in sample_corpas:
  output_corpus += m.parse(sentence) + "\n"
print("\n【分かち書き後の学習データ】:\n\n",output_corpus)

【生の学習データの例】:

 ['月末に菊間先生も誘って研究室で飲み会をします。多分費用は掛かりませんw', 'この前合コンで大失敗しました。', '話を途切れさせてはまずいと焦って自分の話しかしなかったのがいけなかったな', '橋本君合コンの設定とかしてくれない？']

【分かち書き後の学習データ】:

 月末 に 菊間 先生 も 誘っ て 研究室 で 飲み 会 を し ます 。 多分 費用 は 掛かり ませ ん w 

この 前 合コン で 大 失敗 し まし た 。 

話 を 途切れ させ て は まずい と 焦っ て 自分 の 話 しか し なかっ た の が いけ なかっ た な 

橋本 君 合コン の 設定 とか し て くれ ない ？ 


